In [39]:
import torch
print("Colab GPU available:", torch.cuda.is_available())
# !nvidia-smi

Colab GPU available: True


In [40]:
!nvidia-smi

Fri Nov 28 05:16:22 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   35C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [41]:
!ls

CMakelists.txt	matrix_multiplication  test_colab_server.ipynb
include		README.md	       vector_add


In [43]:
!cmake --version

cmake version 3.31.10

CMake suite maintained and supported by Kitware (kitware.com/cmake).


# Run from here always after any change in git

In [50]:
!rm -rf ./*

In [51]:
!ls

# Make a new clone from the git repo and then run it

In [52]:
# Always start from /content
%cd /content

# Remove any old copies so paths don't get nested
!rm -rf GPU_mode

# Clone your repo (note the exact repo name in the URL)
!git clone https://github.com/parthshinde1221/GPU_mode.git

# Enter the repo root
%cd GPU_mode

# Sanity check: you MUST see CMakeLists.txt here
!ls


/content
Cloning into 'GPU_mode'...
remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 38 (delta 15), reused 32 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (38/38), 7.90 KiB | 7.90 MiB/s, done.
Resolving deltas: 100% (15/15), done.
/content/GPU_mode
CMakeLists.txt	matrix_multiplication  test_colab_server.ipynb
include		README.md	       vector_add


# Building all CUDA kernels

In [53]:
# Configure the project (top-level CMakeLists.txt)
!cmake -S . -B build -DCMAKE_BUILD_TYPE=Release

# Build all targets (matmul, vec_add, etc.)
!cmake --build build -j 4


-- The CXX compiler identification is GNU 11.4.0
-- The CUDA compiler identification is NVIDIA 12.5.82 with host compiler GNU 11.4.0
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Detecting CUDA compiler ABI info
-- Detecting CUDA compiler ABI info - done
-- Check for working CUDA compiler: /usr/local/cuda/bin/nvcc - skipped
-- Detecting CUDA compile features
-- Detecting CUDA compile features - done
-- Configuring done (2.3s)
-- Generating done (0.0s)
-- Build files have been written to: /content/GPU_mode/build
[ 33%] Building CUDA object matrix_multiplication/CMakeFiles/matmul.dir/matmul.cu.o
[ 33%] Building CUDA object vector_add/CMakeFiles/vec_add.dir/vec_add.cu.o
[ 50%] Linking CUDA device code CMakeFiles/vec_add.dir/cmake_device_link.o
[ 66%] Linking CUDA device code CMakeFiles/matmul.dir/cmake_device_link.o
[ 83%] L

# Optional build single kernels

In [ ]:
# kernels = ["matmul", "vec_add"]  # target names from each CMakeLists.txt

# for k in kernels:
#     print(f"\n=== Building {k} ===")
#     !cmake --build build --target {k} -j 4


# Running all Kernels

In [ ]:
kernels = ["matmul_naive", "vec_add_naive","matmul_tiled","vec_add_opt"]  # add more later: "softmax", "conv", etc.

for k in kernels:
    print(f"\n=== Running {k} ===")
    !./build/bin/{k}


=== Running matmul ===
C[0] = 1024 (expected 1024)

=== Running vec_add ===
c[0] = 3 (expected 3)
c[N-1] = 3 (expected 3)


# CUDA MemCheck all kernels

In [64]:
!mkdir -p profiles

In [65]:
!ls

build		include		       profiles   test_colab_server.ipynb
CMakeLists.txt	matrix_multiplication  README.md  vector_add


In [ ]:
kernels = ["matmul_naive", "vec_add_naive","matmul_tiled","vec_add_opt"]

for k in kernels:
    print(f"\n=== cuda-memcheck {k} ===")
    !cuda-memcheck --leak-check full ./build/bin/{k} > profiles/{k}_memcheck.txt 2>&1
    print(f"Saved profiles/{k}_memcheck.txt")



=== cuda-memcheck matmul ===
Saved profiles/matmul_memcheck.txt

=== cuda-memcheck vec_add ===
Saved profiles/vec_add_memcheck.txt


# NCU Profile each kernel all kernels

In [58]:
!ls

build		include		       profiles   test_colab_server.ipynb
CMakeLists.txt	matrix_multiplication  README.md  vector_add


In [ ]:
!ls build

bin		CMakeFiles	     Makefile		    vector_add
CMakeCache.txt	cmake_install.cmake  matrix_multiplication


In [60]:
!ls build/bin

matmul	vec_add


In [ ]:
!ncu build/bin 

==ERROR== 'build/bin/.' does not exist or is not an executable. Please make sure to specify the absolute path to 'build/bin/.' if the executable is not in the local directory.


In [68]:
!ls

build		include		       profiles   test_colab_server.ipynb
CMakeLists.txt	matrix_multiplication  README.md  vector_add


In [ ]:
# from IPython.display import IFrame
# import os

# os.makedirs("profiles", exist_ok=True)

# kernels = ["matmul", "vec_add"]

# for k in kernels:
#     print(f"\n=== Profiling {k} with ncu ===")
#     # NCU_DEFAULTS="" clears any default --export that Colab may set
#     !NCU_DEFAULTS="" ncu -f --set full --export html -o profiles/{k} ./build/bin/{k}
#     display(IFrame(f"profiles/{k}.html", width=1024, height=600))


In [ ]:
import os

os.makedirs("profiles", exist_ok=True)

kernels = ["matmul_naive", "vec_add_naive","matmul_tiled","vec_add_opt"]

# -------- First loop: run profiling & save reports --------
print("=== Running Nsight Compute and saving reports ===")
for k in kernels:
    print(f"\n[1] Profiling {k} with ncu ...")
    !NCU_DEFAULTS="" ncu -f --set full -o profiles/{k} ./build/bin/{k}
    print(f"--> Saved report: profiles/{k}.ncu-rep")
    
    # Print the kernel Duration from the report
    print(f"--> Duration for {k}:")
    !NCU_DEFAULTS="" ncu --import profiles/{k}.ncu-rep --page details | grep "Duration"

print("\nAll reports saved:")
!ls -lh profiles


=== Running Nsight Compute and saving reports ===

[1] Profiling matmul with ncu ...
==PROF== Connected to process 44603 (/content/GPU_mode/build/bin/matmul)
==PROF== Profiling "matmul_kernel" - 0: 0%....50%....100% - 30 passes
C[0] = 1024 (expected 1024)
==PROF== Disconnected from process 44603
==PROF== Report: /content/GPU_mode/profiles/matmul.ncu-rep
--> Saved report: profiles/matmul.ncu-rep
--> Duration for matmul:
    Duration                         ms         1.15

[1] Profiling vec_add with ncu ...
==PROF== Connected to process 44700 (/content/GPU_mode/build/bin/vec_add)
==PROF== Profiling "vec_add_kernel" - 0: 0%....50%....100% - 30 passes
c[0] = 3 (expected 3)
c[N-1] = 3 (expected 3)
==PROF== Disconnected from process 44700
==PROF== Report: /content/GPU_mode/profiles/vec_add.ncu-rep
--> Saved report: profiles/vec_add.ncu-rep
--> Duration for vec_add:
    Duration                         us        49.95

All reports saved:
total 464K
-rw-r--r-- 1 root root   52 Nov 28 05:46 ma

In [77]:
# -------- Second loop: re-import reports and show details --------
print("\n=== Printing Nsight Compute details for each kernel ===")
for k in kernels:
    print(f"\n[2] Nsight Compute details page for {k}")
    !NCU_DEFAULTS="" ncu --import profiles/{k}.ncu-rep --page details


=== Printing Nsight Compute details for each kernel ===

[2] Nsight Compute details page for matmul
[40910] matmul@127.0.0.1
  matmul_kernel(const float *, const float *, float *, int) (32, 32, 1)x(16, 16, 1), Context 1, Stream 7, Device 0, CC 7.5
    Section: GPU Speed Of Light Throughput
    ----------------------- ----------- ------------
    Metric Name             Metric Unit Metric Value
    ----------------------- ----------- ------------
    DRAM Frequency                  Ghz         5.00
    SM Frequency                    Mhz       584.97
    Elapsed Cycles                cycle      672,698
    Memory Throughput                 %        62.40
    DRAM Throughput                   %         1.26
    Duration                         ms         1.15
    L1/TEX Cache Throughput           %        93.62
    L2 Cache Throughput               %         6.75
    SM Active Cycles              cycle   660,623.18
    Compute (SM) Throughput           %        62.40
    ---------------

In [ ]:
# print("Running matmul:")
# !./build/bin/matmul

# print("\nRunning vec_add:")
# !./build/bin/vec_add
